In [79]:
# Import required libraries and dependencies
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

import numpy as np
from scipy.stats import linregress

import matplotlib.pyplot as plt
import hvplot.pandas

In [2]:
### import and read csv file

path = 'Output/full_indicators_by_country_by_year.csv'
full_indicators_df = pd.read_csv(path)
# full_indicators = full_indicators.rename(columns={'total':'Immigration_flow'})

full_indicators_df.head(5)

,index,country,iso3Code,year,clean fuels for cooking (% of population),Access to electricity (% of population),Agricultural land (% ),Armed forces (% labor force),precipitation (mm per year),"Birth rate, crude (per 1,000 people)",...,"Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)",Total debt service (% of GNI),"Total debt service (% of exports of goods, services and primary income)",Unemployment advanced education,Unemployment basic education,Unemployment intermediate education,"Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,immigration_100k
0,0,Afghanistan,AFG,2015,27.6,71.5,58.123668,3.724667,327.0,38.803,...,0.009,0.292137,3.299608,0.00,0.000,0.000,9.052,20.0,3.294,7.758981
1,1,Afghanistan,AFG,2016,28.8,97.7,58.123668,3.635915,327.0,37.936,...,0.013,0.324903,4.333403,0.00,0.000,0.000,10.133,20.0,3.294,7.651162
2,2,Afghanistan,AFG,2017,30.3,97.7,58.123668,3.501771,327.0,37.342,...,0.017,0.333475,4.353572,15.46,12.283,16.155,11.184,20.0,3.294,9.694882
3,3,Afghanistan,AFG,2018,31.4,93.4,58.276988,2.961201,327.0,36.927,...,0.016,0.344999,3.323800,15.46,12.283,16.155,11.196,20.0,3.294,9.688910
4,4,Afghanistan,AFG,2019,32.6,97.7,58.276988,3.023536,327.0,36.466,...,0.016,0.291267,2.974238,15.46,12.283,16.155,11.185,20.0,3.294,10.275749


In [3]:
# Generate summary statistics
full_indicators_df.describe()

,index,year,clean fuels for cooking (% of population),Access to electricity (% of population),Agricultural land (% ),Armed forces (% labor force),precipitation (mm per year),"Birth rate, crude (per 1,000 people)","Central government debt, total (% of GDP)",Consumer price index (2010 = 100),...,"Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)",Total debt service (% of GNI),"Total debt service (% of exports of goods, services and primary income)",Unemployment advanced education,Unemployment basic education,Unemployment intermediate education,"Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,immigration_100k
count,1663.000000,1663.000000,1663.000000,1663.000000,1663.000000,1663.000000,1663.000000,1663.000000,1663.000000,1663.000000,...,1663.000000,1663.000000,1663.000000,1663.000000,1663.000000,1663.000000,1663.000000,1663.000000,1663.000000,1663.000000
mean,831.000000,2019.034275,65.258328,83.683824,38.149248,1.140878,1080.260313,19.438815,23.912632,315.239622,...,5.118244,3.461613,9.987065,5.576078,6.920728,6.748378,7.088161,40.164161,1.853091,11.171426
std,480.211065,2.589557,39.562522,27.096966,22.524579,1.549332,803.647910,10.131943,39.928103,1857.934281,...,4.046082,5.889656,13.997079,5.854945,7.703278,6.619296,5.775616,21.050826,0.886108,34.368195
min,0.000000,2015.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,415.500000,2017.000000,25.600000,73.750000,19.602978,0.336253,460.000000,10.800000,0.000000,112.430438,...,1.420000,0.000000,0.000000,1.293000,0.480500,1.236500,3.272000,25.000000,1.564000,0.721311
50%,831.000000,2019.000000,87.500000,99.900000,39.434768,0.682330,900.000000,16.833000,0.000000,127.508994,...,4.380000,1.263797,5.002966,4.040000,4.631000,5.490000,5.255000,37.000000,2.028000,2.396329
75%,1246.500000,2021.000000,100.000000,100.000000,54.637271,1.342953,1622.000000,27.179000,44.000660,159.761076,...,8.315000,4.665517,14.570040,7.716000,10.420000,9.767500,9.498500,54.000000,2.332000,7.089806
max,1662.000000,2023.000000,100.000000,100.000000,84.563658,14.208087,3240.000000,46.558000,249.411354,38796.557297,...,16.990000,88.963244,127.580662,33.291000,46.530000,38.182000,34.007000,90.000000,3.397000,448.765894


---

### Prepare the Data

In [4]:
full_indicators_df.columns

Index(['index', 'country', 'iso3Code', 'year',
       'clean fuels for cooking (% of population)',
       'Access to electricity (% of population)', 'Agricultural land (% )',
       'Armed forces (% labor force)', 'precipitation (mm per year)',
       'Birth rate, crude (per 1,000 people)',
       'Central government debt, total (% of GDP)',
       'Consumer price index (2010 = 100)', 'Control of Corruption: Estimate',
       'Cost of business start-up (% of GNI per capita)',
       'Current account balance (% of GDP)',
       'Current health expenditure (% of GDP)',
       'Death rate, crude (per 1,000 people)',
       'Debt service (PPG and IMF only, % of exports of goods, services and primary income)',
       'doing business score', 'Fertility rate, total (births per woman)',
       '% Firms using banks', 'GDP growth (annual %)',
       'GDP per capita (constant 2015 US$)',
       'GDP per capita, PPP (constant 2021 international $)',
       'GNI per capita (2015 US$)', 'government 

### Normalize the data using StandardScaler()

In [6]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file

indicators_scaled = StandardScaler().fit_transform(full_indicators_df[[

                                                                      'clean fuels for cooking (% of population)',
                                                                      'Access to electricity (% of population)', 'Agricultural land (% )',
                                                                      'Armed forces (% labor force)', 'precipitation (mm per year)',
                                                                      'Birth rate, crude (per 1,000 people)',
                                                                      'Central government debt, total (% of GDP)',
                                                                      'Consumer price index (2010 = 100)', 'Control of Corruption: Estimate',
                                                                      'Cost of business start-up (% of GNI per capita)',
                                                                      'Current account balance (% of GDP)',
                                                                      'Current health expenditure (% of GDP)',
                                                                      'Death rate, crude (per 1,000 people)',
                                                                      'Debt service (PPG and IMF only, % of exports of goods, services and primary income)',
                                                                      'doing business score', 'Fertility rate, total (births per woman)',
                                                                      '% Firms using banks', 'GDP growth (annual %)',
                                                                      'GDP per capita (constant 2015 US$)',
                                                                      'GDP per capita, PPP (constant 2021 international $)',
                                                                      'GNI per capita (2015 US$)', 'government  consumption exp (% of GDP)',
                                                                      'Government expenditure on education, total (% of GDP)',
                                                                      'Human Capital Index',
                                                                      'Industry (including construction), value added (% of GDP)',
                                                                      'Inflation, consumer prices (annual %)', 'Level of water stress',
                                                                      'Life expectancy at birth, total (years)',
                                                                      'Military expenditure (% of GDP)', 'Population density',
                                                                      'Population growth (annual %)', '% of total population below 5 meters',
                                                                      'Population living in slums (% of urban population)',
                                                                      'Population, total', 'Poverty $2.15 a day (% of population)',
                                                                      'Poverty $3.65 a day (% of population)',
                                                                      'Poverty $6.85 a day (% of population)', '%  Poverty headcount ratio ',
                                                                      'Time to start a business (days)',
                                                                      'Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)',
                                                                      'Total debt service (% of GNI)',
                                                                      'Total debt service (% of exports of goods, services and primary income)',
                                                                      'Unemployment advanced education', 'Unemployment basic education ',
                                                                      'Unemployment intermediate education',
                                                                      'Unemployment, total (% of total labor force) (modeled ILO estimate)',
                                                                      'Corruption index 2023', 'Global_Peace_Index_2023', 'immigration_100k'
    

                                                                      
                                                                      ]])

In [8]:
# Create a DataFrame with the scaled data
indicators_scaled_df = pd.DataFrame(indicators_scaled, columns=[
                                                               
                                                               'clean fuels for cooking (% of population)',
                                                               'Access to electricity (% of population)', 'Agricultural land (% )',
                                                               'Armed forces (% labor force)', 'precipitation (mm per year)',
                                                               'Birth rate, crude (per 1,000 people)',
                                                               'Central government debt, total (% of GDP)',
                                                               'Consumer price index (2010 = 100)', 'Control of Corruption: Estimate',
                                                               'Cost of business start-up (% of GNI per capita)',
                                                               'Current account balance (% of GDP)',
                                                               'Current health expenditure (% of GDP)',
                                                               'Death rate, crude (per 1,000 people)',
                                                               'Debt service (PPG and IMF only, % of exports of goods, services and primary income)',
                                                               'doing business score', 'Fertility rate, total (births per woman)',
                                                               '% Firms using banks', 'GDP growth (annual %)',
                                                               'GDP per capita (constant 2015 US$)',
                                                               'GDP per capita, PPP (constant 2021 international $)',
                                                               'GNI per capita (2015 US$)', 'government  consumption exp (% of GDP)',
                                                               'Government expenditure on education, total (% of GDP)',
                                                               'Human Capital Index',
                                                               'Industry (including construction), value added (% of GDP)',
                                                               'Inflation, consumer prices (annual %)', 'Level of water stress',
                                                               'Life expectancy at birth, total (years)',
                                                               'Military expenditure (% of GDP)', 'Population density',
                                                               'Population growth (annual %)', '% of total population below 5 meters',
                                                               'Population living in slums (% of urban population)',
                                                               'Population, total', 'Poverty $2.15 a day (% of population)',
                                                               'Poverty $3.65 a day (% of population)',
                                                               'Poverty $6.85 a day (% of population)', '%  Poverty headcount ratio ',
                                                               'Time to start a business (days)',
                                                               'Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)',
                                                               'Total debt service (% of GNI)',
                                                               'Total debt service (% of exports of goods, services and primary income)',
                                                               'Unemployment advanced education', 'Unemployment basic education ',
                                                               'Unemployment intermediate education',
                                                               'Unemployment, total (% of total labor force) (modeled ILO estimate)',
                                                               'Corruption index 2023', 'Global_Peace_Index_2023', 'immigration_100k'
                                                        
                                                               ])

# # Copy the crypto names from the original DataFrame

indicators_scaled_df['country-year'] = full_indicators_df['country'].astype(str) + ' - ' + full_indicators_df['year'].astype(str)



# Display the scaled DataFrame
indicators_scaled_df.head()

,clean fuels for cooking (% of population),Access to electricity (% of population),Agricultural land (% ),Armed forces (% labor force),precipitation (mm per year),"Birth rate, crude (per 1,000 people)","Central government debt, total (% of GDP)",Consumer price index (2010 = 100),Control of Corruption: Estimate,Cost of business start-up (% of GNI per capita),...,Total debt service (% of GNI),"Total debt service (% of exports of goods, services and primary income)",Unemployment advanced education,Unemployment basic education,Unemployment intermediate education,"Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,immigration_100k,country-year
0,-0.952155,-0.449773,0.887050,1.668181,-0.937583,1.911776,-0.599072,-0.098180,-1.247837,-0.071469,...,-0.538305,-0.477919,-0.952657,-0.898684,-1.019808,0.340125,-0.958168,1.626599,-0.099321,Afghanistan - 2015
1,-0.921814,0.517416,0.887050,1.610880,-0.937583,1.826180,-0.599072,-0.095043,-1.430819,-0.048522,...,-0.532740,-0.404039,-0.952657,-0.898684,-1.019808,0.527347,-0.958168,1.626599,-0.102459,Afghanistan - 2016
2,-0.883888,0.517416,0.887050,1.524272,-0.937583,1.767536,-0.599072,-0.091327,-1.420804,1.542467,...,-0.531284,-0.402597,1.688640,0.696312,1.421518,0.709374,-0.958168,1.626599,-0.042975,Afghanistan - 2017
3,-0.856076,0.358678,0.893859,1.175262,-0.937583,1.726564,-0.599072,-0.090836,-1.393977,-0.392726,...,-0.529327,-0.476190,1.688640,0.696312,1.421518,0.711452,-0.958168,1.626599,-0.043149,Afghanistan - 2018
4,-0.825735,0.517416,0.893859,1.215508,-0.937583,1.681050,-0.599072,-0.089020,-1.311552,-0.382527,...,-0.538452,-0.501171,1.688640,0.696312,1.421518,0.709547,-0.958168,1.626599,-0.026069,Afghanistan - 2019


In [9]:
## Rename the indicators to be more ... friendly

indicators_scaled_df = indicators_scaled_df.rename(columns={


                                            'Access to clean fuels and technologies for cooking (% of population)'    :   'clean fuels for cooking (% of population)'  ,
                                            'Access to electricity (% of population)'    :   'Access to electricity (% of population)'  ,
                                            'Agricultural land (% of land area)'    :   'Agricultural land (% )'  ,
                                            'Armed forces personnel (% of total labor force)'    :   'Armed forces (% labor force)'  ,
                                            'Average precipitation in depth (mm per year)'    :   'precipitation (mm per year)'  ,
                                            'Birth rate crude (per 1000 people)'    :   'Birth rate crude'  ,
                                            'Central government debt total (% of GDP)'    :   'government debt (% of GDP)'  ,
                                            'Consumer price index (2010 = 100) Control of Corruption: Estimate'    :   'Consumer price index (2010)'  ,
                                            'Cost of business start-up procedures (% of GNI per capita)'    :   'Cost of business start-up (% of GNI per capita)'  ,
                                            'Current account balance (% of GDP)'    :   'Current account balance (% of GDP)'  ,
                                            'Current health expenditure (% of GDP)'    :   'Current health expenditure (% of GDP)'  ,
                                            'Death rate crude (per 1000 people)'    :   'Death rate crude (per 1000 people)'  ,
                                            'Debt service (PPG and IMF only % of exports of goods services and primary income)'    :   'Debt service'  ,
                                            'Ease of doing business score (0 = lowest performance to 100 = best performance)'    :   'doing business score'  ,
                                            'Fertility rate total (births per woman)'    :   'Fertility rate per woman'  ,
                                            'Firms using banks to finance investment (% of firms)'    :   '% Firms using banks'  ,
                                            'GDP growth (annual %) GDP per capita (constant 2015 US$)'    :   'GDP per capita growth '  ,
                                            'GDP per capita PPP (constant 2021 international $)'    :   'GDP per capita PPP ( 2021)'  ,
                                            'GNI per capita (constant 2015 US$)'    :   'GNI per capita (2015 US$)'  ,
                                            'General government final consumption expenditure (% of GDP)'    :   'government  consumption exp (% of GDP)'  ,
                                            'Government expenditure on education total (% of GDP)'    :   'Government expenditure on education total (% of GDP)'  ,
                                            'Human Capital Index (HCI) (scale 0-1)'    :   'Human Capital Index'  ,
                                            'Industry (including construction) value added (% of GDP)'    :   'Industry (% of GDP)'  ,
                                            'Inflation consumer prices (annual %)'    :   'Inflation consumer prices (%)'  ,
                                            'Level of water stress: freshwater withdrawal as a proportion of available freshwater resources'    :   'Level of water stress'  ,
                                            'Life expectancy at birth total (years)'    :   'Life expectancy at birth total (years)'  ,
                                            'Military expenditure (% of GDP)'    :   'Military expenditure (% of GDP)'  ,
                                            'Population density (people per sq. km of land area)'    :   'Population density'  ,
                                            'Population growth (annual %)'    :   'Population growth (annual %)'  ,
                                            'Population living in areas where elevation is below 5 meters (% of total population)'    :   '% of total population below 5 meters'  ,
                                            'Population living in slums (% of urban population)'    :   'Population living in slums (% of urban population)'  ,
                                            'Population total'    :   'Population total'  ,
                                            'Poverty headcount ratio at $2.15 a day (2017 PPP) (% of population)'    :   'Poverty $2.15 a day (% of population)'  ,
                                            'Poverty headcount ratio at $3.65 a day (2017 PPP) (% of population)'    :   'Poverty $3.65 a day (% of population)'  ,
                                            'Poverty headcount ratio at $6.85 a day (2017 PPP) (% of population)'    :   'Poverty $6.85 a day (% of population)'  ,
                                            'Poverty headcount ratio at national poverty lines (% of population)'    :   '%  Poverty headcount ratio '  ,
                                            'Time required to start a business (days)'    :   'Time to start a business (days)'  ,
                                            'Total alcohol consumption per capita (liters of pure alcohol projected estimates 15+ years of age)'    :   'Alcohol consumption per capita'  ,
                                            'Total debt service (% of GNI)'    :   'Total debt service (% of GNI)'  ,
                                            'Total debt service (% of exports of goods services and primary income)'    :   'Total debt service (% of exports)'  ,
                                            'Unemployment with advanced education (% of total labor force with advanced education)'    :   'Unemployment advanced education'  ,
                                            'Unemployment with basic education (% of total labor force with basic education)'    :   'Unemployment basic education '  ,
                                            'Unemployment with intermediate education (% of total labor force with intermediate education)'    :   'Unemployment intermediate education'  ,
                                            'Unemployment total (% of total labor force) (modeled ILO estimate)]'    :   'Unemployment total'  ,
                                            'CPI_score_2023' : 'Corruption index 2023'


                                                    })
indicators_scaled_df.head(5)

,clean fuels for cooking (% of population),Access to electricity (% of population),Agricultural land (% ),Armed forces (% labor force),precipitation (mm per year),"Birth rate, crude (per 1,000 people)","Central government debt, total (% of GDP)",Consumer price index (2010 = 100),Control of Corruption: Estimate,Cost of business start-up (% of GNI per capita),...,Total debt service (% of GNI),"Total debt service (% of exports of goods, services and primary income)",Unemployment advanced education,Unemployment basic education,Unemployment intermediate education,"Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,immigration_100k,country-year
0,-0.952155,-0.449773,0.887050,1.668181,-0.937583,1.911776,-0.599072,-0.098180,-1.247837,-0.071469,...,-0.538305,-0.477919,-0.952657,-0.898684,-1.019808,0.340125,-0.958168,1.626599,-0.099321,Afghanistan - 2015
1,-0.921814,0.517416,0.887050,1.610880,-0.937583,1.826180,-0.599072,-0.095043,-1.430819,-0.048522,...,-0.532740,-0.404039,-0.952657,-0.898684,-1.019808,0.527347,-0.958168,1.626599,-0.102459,Afghanistan - 2016
2,-0.883888,0.517416,0.887050,1.524272,-0.937583,1.767536,-0.599072,-0.091327,-1.420804,1.542467,...,-0.531284,-0.402597,1.688640,0.696312,1.421518,0.709374,-0.958168,1.626599,-0.042975,Afghanistan - 2017
3,-0.856076,0.358678,0.893859,1.175262,-0.937583,1.726564,-0.599072,-0.090836,-1.393977,-0.392726,...,-0.529327,-0.476190,1.688640,0.696312,1.421518,0.711452,-0.958168,1.626599,-0.043149,Afghanistan - 2018
4,-0.825735,0.517416,0.893859,1.215508,-0.937583,1.681050,-0.599072,-0.089020,-1.311552,-0.382527,...,-0.538452,-0.501171,1.688640,0.696312,1.421518,0.709547,-0.958168,1.626599,-0.026069,Afghanistan - 2019


In [10]:
indicators_scaled_df.columns

Index(['clean fuels for cooking (% of population)',
       'Access to electricity (% of population)', 'Agricultural land (% )',
       'Armed forces (% labor force)', 'precipitation (mm per year)',
       'Birth rate, crude (per 1,000 people)',
       'Central government debt, total (% of GDP)',
       'Consumer price index (2010 = 100)', 'Control of Corruption: Estimate',
       'Cost of business start-up (% of GNI per capita)',
       'Current account balance (% of GDP)',
       'Current health expenditure (% of GDP)',
       'Death rate, crude (per 1,000 people)',
       'Debt service (PPG and IMF only, % of exports of goods, services and primary income)',
       'doing business score', 'Fertility rate, total (births per woman)',
       '% Firms using banks', 'GDP growth (annual %)',
       'GDP per capita (constant 2015 US$)',
       'GDP per capita, PPP (constant 2021 international $)',
       'GNI per capita (2015 US$)', 'government  consumption exp (% of GDP)',
       'Governmen

In [12]:
# Organize columns

indicators_scaled_df = indicators_scaled_df[[

                                                'country-year', 
                                                'clean fuels for cooking (% of population)',
                                                'Access to electricity (% of population)', 'Agricultural land (% )',
                                            'Armed forces (% labor force)', 'precipitation (mm per year)',
                                            'Birth rate, crude (per 1,000 people)',
                                            'Central government debt, total (% of GDP)',
                                            'Consumer price index (2010 = 100)', 'Control of Corruption: Estimate',
                                            'Cost of business start-up (% of GNI per capita)',
                                            'Current account balance (% of GDP)',
                                            'Current health expenditure (% of GDP)',
                                            'Death rate, crude (per 1,000 people)',
                                            'Debt service (PPG and IMF only, % of exports of goods, services and primary income)',
                                            'doing business score', 'Fertility rate, total (births per woman)',
                                            '% Firms using banks', 'GDP growth (annual %)',
                                            'GDP per capita (constant 2015 US$)',
                                            'GDP per capita, PPP (constant 2021 international $)',
                                            'GNI per capita (2015 US$)', 'government  consumption exp (% of GDP)',
                                            'Government expenditure on education, total (% of GDP)',
                                            'Human Capital Index',
                                            'Industry (including construction), value added (% of GDP)',
                                            'Inflation, consumer prices (annual %)', 'Level of water stress',
                                            'Life expectancy at birth, total (years)',
                                            'Military expenditure (% of GDP)', 'Population density',
                                            'Population growth (annual %)', '% of total population below 5 meters',
                                            'Population living in slums (% of urban population)',
                                            'Population, total', 'Poverty $2.15 a day (% of population)',
                                            'Poverty $3.65 a day (% of population)',
                                            'Poverty $6.85 a day (% of population)', '%  Poverty headcount ratio ',
                                            'Time to start a business (days)',
                                            'Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)',
                                            'Total debt service (% of GNI)',
                                            'Total debt service (% of exports of goods, services and primary income)',
                                            'Unemployment advanced education', 'Unemployment basic education ',
                                            'Unemployment intermediate education',
                                            'Unemployment, total (% of total labor force) (modeled ILO estimate)',
                                            'Corruption index 2023', 'Global_Peace_Index_2023', 'immigration_100k'
                                                                
                                            ]]
indicators_scaled_df.head(5)

,country-year,clean fuels for cooking (% of population),Access to electricity (% of population),Agricultural land (% ),Armed forces (% labor force),precipitation (mm per year),"Birth rate, crude (per 1,000 people)","Central government debt, total (% of GDP)",Consumer price index (2010 = 100),Control of Corruption: Estimate,...,"Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)",Total debt service (% of GNI),"Total debt service (% of exports of goods, services and primary income)",Unemployment advanced education,Unemployment basic education,Unemployment intermediate education,"Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,immigration_100k
0,Afghanistan - 2015,-0.952155,-0.449773,0.887050,1.668181,-0.937583,1.911776,-0.599072,-0.098180,-1.247837,...,-1.263143,-0.538305,-0.477919,-0.952657,-0.898684,-1.019808,0.340125,-0.958168,1.626599,-0.099321
1,Afghanistan - 2016,-0.921814,0.517416,0.887050,1.610880,-0.937583,1.826180,-0.599072,-0.095043,-1.430819,...,-1.262154,-0.532740,-0.404039,-0.952657,-0.898684,-1.019808,0.527347,-0.958168,1.626599,-0.102459
2,Afghanistan - 2017,-0.883888,0.517416,0.887050,1.524272,-0.937583,1.767536,-0.599072,-0.091327,-1.420804,...,-1.261165,-0.531284,-0.402597,1.688640,0.696312,1.421518,0.709374,-0.958168,1.626599,-0.042975
3,Afghanistan - 2018,-0.856076,0.358678,0.893859,1.175262,-0.937583,1.726564,-0.599072,-0.090836,-1.393977,...,-1.261413,-0.529327,-0.476190,1.688640,0.696312,1.421518,0.711452,-0.958168,1.626599,-0.043149
4,Afghanistan - 2019,-0.825735,0.517416,0.893859,1.215508,-0.937583,1.681050,-0.599072,-0.089020,-1.311552,...,-1.261413,-0.538452,-0.501171,1.688640,0.696312,1.421518,0.709547,-0.958168,1.626599,-0.026069


In [13]:
indicators_scaled_df = indicators_scaled_df.set_index(['country-year'])
indicators_scaled_df.head(5)

,clean fuels for cooking (% of population),Access to electricity (% of population),Agricultural land (% ),Armed forces (% labor force),precipitation (mm per year),"Birth rate, crude (per 1,000 people)","Central government debt, total (% of GDP)",Consumer price index (2010 = 100),Control of Corruption: Estimate,Cost of business start-up (% of GNI per capita),...,"Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)",Total debt service (% of GNI),"Total debt service (% of exports of goods, services and primary income)",Unemployment advanced education,Unemployment basic education,Unemployment intermediate education,"Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,immigration_100k
country-year,,,,,,,,,,,,,,,,,,,,,
Afghanistan - 2015,-0.952155,-0.449773,0.887050,1.668181,-0.937583,1.911776,-0.599072,-0.098180,-1.247837,-0.071469,...,-1.263143,-0.538305,-0.477919,-0.952657,-0.898684,-1.019808,0.340125,-0.958168,1.626599,-0.099321
Afghanistan - 2016,-0.921814,0.517416,0.887050,1.610880,-0.937583,1.826180,-0.599072,-0.095043,-1.430819,-0.048522,...,-1.262154,-0.532740,-0.404039,-0.952657,-0.898684,-1.019808,0.527347,-0.958168,1.626599,-0.102459
Afghanistan - 2017,-0.883888,0.517416,0.887050,1.524272,-0.937583,1.767536,-0.599072,-0.091327,-1.420804,1.542467,...,-1.261165,-0.531284,-0.402597,1.688640,0.696312,1.421518,0.709374,-0.958168,1.626599,-0.042975
Afghanistan - 2018,-0.856076,0.358678,0.893859,1.175262,-0.937583,1.726564,-0.599072,-0.090836,-1.393977,-0.392726,...,-1.261413,-0.529327,-0.476190,1.688640,0.696312,1.421518,0.711452,-0.958168,1.626599,-0.043149
Afghanistan - 2019,-0.825735,0.517416,0.893859,1.215508,-0.937583,1.681050,-0.599072,-0.089020,-1.311552,-0.382527,...,-1.261413,-0.538452,-0.501171,1.688640,0.696312,1.421518,0.709547,-0.958168,1.626599,-0.026069


---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [14]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,12))
k

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [15]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list

for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(indicators_scaled_df)
    inertia.append(k_model.inertia_)


In [16]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)
df_elbow.head(12)

,k,inertia
0,1,81487.000000
1,2,69539.341081
2,3,67894.214661
3,4,62578.421353
4,5,59612.120303
5,6,55812.891482
6,7,53479.023491
7,8,53194.046248
8,9,50659.346194
9,10,48394.421218


In [17]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
elbow_original_data = df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve without PCA", 
    xticks=k
)

elbow_original_data


:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:**  4

---

### Cluster the immigration data with K-means Using the Original Scaled DataFrame

In [18]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=0)

In [19]:
# Fit the K-Means model using the scaled DataFrame
model.fit(indicators_scaled_df)

KMeans(n_clusters=4, random_state=0)

In [22]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
kmeans_predictions = model.predict(indicators_scaled_df)

# Print the resulting array of cluster values.
print(kmeans_predictions)

[1 1 1 ... 1 3 3]


In [23]:
# Create a copy of the scaled DataFrame
indicators_prediction_df = indicators_scaled_df.copy()

In [24]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
indicators_prediction_df['countries_cluster_original_data'] = kmeans_predictions

# Display the copy of the scaled DataFrame
indicators_prediction_df.head()

,clean fuels for cooking (% of population),Access to electricity (% of population),Agricultural land (% ),Armed forces (% labor force),precipitation (mm per year),"Birth rate, crude (per 1,000 people)","Central government debt, total (% of GDP)",Consumer price index (2010 = 100),Control of Corruption: Estimate,Cost of business start-up (% of GNI per capita),...,Total debt service (% of GNI),"Total debt service (% of exports of goods, services and primary income)",Unemployment advanced education,Unemployment basic education,Unemployment intermediate education,"Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,immigration_100k,countries_cluster_original_data
country-year,,,,,,,,,,,,,,,,,,,,,
Afghanistan - 2015,-0.952155,-0.449773,0.887050,1.668181,-0.937583,1.911776,-0.599072,-0.098180,-1.247837,-0.071469,...,-0.538305,-0.477919,-0.952657,-0.898684,-1.019808,0.340125,-0.958168,1.626599,-0.099321,1
Afghanistan - 2016,-0.921814,0.517416,0.887050,1.610880,-0.937583,1.826180,-0.599072,-0.095043,-1.430819,-0.048522,...,-0.532740,-0.404039,-0.952657,-0.898684,-1.019808,0.527347,-0.958168,1.626599,-0.102459,1
Afghanistan - 2017,-0.883888,0.517416,0.887050,1.524272,-0.937583,1.767536,-0.599072,-0.091327,-1.420804,1.542467,...,-0.531284,-0.402597,1.688640,0.696312,1.421518,0.709374,-0.958168,1.626599,-0.042975,1
Afghanistan - 2018,-0.856076,0.358678,0.893859,1.175262,-0.937583,1.726564,-0.599072,-0.090836,-1.393977,-0.392726,...,-0.529327,-0.476190,1.688640,0.696312,1.421518,0.711452,-0.958168,1.626599,-0.043149,1
Afghanistan - 2019,-0.825735,0.517416,0.893859,1.215508,-0.937583,1.681050,-0.599072,-0.089020,-1.311552,-0.382527,...,-0.538452,-0.501171,1.688640,0.696312,1.421518,0.709547,-0.958168,1.626599,-0.026069,1


In [26]:
# Create a scatter plot using hvPlot by setting
    # `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
    # Color the graph points with the labels found using K-Means and
    # add the crypto name in the `hover_cols` parameter to identify
    # the cryptocurrency represented by each data point.

clusters_original_data = indicators_prediction_df.hvplot.scatter(
    x="immigration_100k", 
    y="Global_Peace_Index_2023", 
    by="countries_cluster_original_data",
    title = "Clusters with original data",
    hover_cols = 'country-year'
)

clusters_original_data

:NdOverlay   [countries_cluster_original_data]
   :Scatter   [immigration_100k]   (Global_Peace_Index_2023,country-year)

---

### Optimize Clusters with Principal Component Analysis.

In [27]:
# Create a PCA model instance and set `n_components=5`.
pca = PCA(n_components=5)

In [28]:
# Use the PCA model with `fit_transform` to reduce the original scaled DataFrame
# down to four principal components.
indicators_pca = pca.fit_transform(indicators_scaled_df)

# View the scaled PCA data
indicators_pca[0:5]

array([[ 3.27020581, -1.97997571, -0.99661033,  0.75322919, -0.33903367],
       [ 3.92600645, -0.82085159, -0.01775231,  0.19169539, -0.76194156],
       [ 4.00193096,  1.25005058, -0.78806944,  0.37849199, -2.52465574],
       [ 3.54891034,  1.61980276, -0.49000297,  0.4737365 , -2.10725466],
       [ 3.41064539,  1.62411933, -0.45960017,  0.55350187, -2.06732118]])

In [29]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
pca.explained_variance_ratio_

array([0.20773766, 0.08690068, 0.06621939, 0.04976276, 0.04811346])

In [30]:
# Use 'sum' on the explained varaibles array to obtain the total explained variance

total_explained_variance = sum(pca.explained_variance_ratio_)
print(f'The total explained variance is : {total_explained_variance}')

The total explained variance is : 0.45873395541721945


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 45.9%

In [32]:
# Create a new DataFrame with the PCA data.
indicators_pca_df = pd.DataFrame(
    indicators_pca,
    columns=["PCA1", "PCA2", 'PCA3', 'PCA4', 'PCA5']
)

# Copy the crypto names from the original scaled DataFrame
indicators_pca_df['country-year'] = full_indicators_df['country'].astype(str) + ' - ' + full_indicators_df['year'].astype(str)

# # Set the coin_id column as index
indicators_pca_df = indicators_pca_df.set_index(['country-year'])

# Display the scaled PCA DataFrame
indicators_pca_df.head(5)

,PCA1,PCA2,PCA3,PCA4,PCA5
country-year,,,,,
Afghanistan - 2015,3.270206,-1.979976,-0.996610,0.753229,-0.339034
Afghanistan - 2016,3.926006,-0.820852,-0.017752,0.191695,-0.761942
Afghanistan - 2017,4.001931,1.250051,-0.788069,0.378492,-2.524656
Afghanistan - 2018,3.548910,1.619803,-0.490003,0.473736,-2.107255
Afghanistan - 2019,3.410645,1.624119,-0.459600,0.553502,-2.067321


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [33]:
# Create a list with the number of k-values from 1 to 11
    # we already have the list
k

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [34]:
# Create an empty list to store the inertia values

inertia_pca = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list

for i in k:
    k_model_pca = KMeans(n_clusters=i, random_state=1)
    k_model_pca.fit(indicators_pca_df)
    inertia_pca.append(k_model_pca.inertia_)

In [35]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_pca = {"k": k, "inertia": inertia_pca}

# Create a DataFrame with the data to plot the Elbow curve

df_elbow_pca = pd.DataFrame(elbow_data_pca)
df_elbow_pca.head()


,k,inertia
0,1,37380.853825
1,2,25532.164918
2,3,20374.028544
3,4,16985.304591
4,5,14900.773142


In [37]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.

elbow_with_pca = df_elbow_pca.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve_with_PCA", 
    xticks=k
)

elbow_with_pca

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No

### Cluster Immigration data with K-means Using the Scaled PCA DataFrame

In [38]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=0)

In [40]:
# Fit the K-Means model using the PCA data
model.fit(indicators_pca_df)

KMeans(n_clusters=4, random_state=0)

In [41]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
kmeans_predictions_pca = model.predict(indicators_pca_df)

# Print the resulting array of cluster values.
print(kmeans_predictions_pca)

[3 3 2 ... 2 2 2]


In [42]:
# Create a copy of the scaled PCA DataFrame

indicators_pca_new_df = indicators_pca_df.copy()

# Add a new column to the copy of the PCA DataFrame with the predicted clusters
indicators_pca_new_df['countries_cluster_withPCA'] = kmeans_predictions_pca

# Display the copy of the scaled PCA DataFrame
indicators_pca_new_df.head(5)

,PCA1,PCA2,PCA3,PCA4,PCA5,countries_cluster_withPCA
country-year,,,,,,
Afghanistan - 2015,3.270206,-1.979976,-0.996610,0.753229,-0.339034,3
Afghanistan - 2016,3.926006,-0.820852,-0.017752,0.191695,-0.761942,3
Afghanistan - 2017,4.001931,1.250051,-0.788069,0.378492,-2.524656,2
Afghanistan - 2018,3.548910,1.619803,-0.490003,0.473736,-2.107255,2
Afghanistan - 2019,3.410645,1.624119,-0.459600,0.553502,-2.067321,2


In [43]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
clusters_with_PCA = indicators_pca_new_df.hvplot.scatter(
    x="PCA1", 
    y="PCA2", 
    by="countries_cluster_withPCA",
    title = "Clusters with PCA" ,
    hover_cols = 'country-year'
)

clusters_with_PCA

:NdOverlay   [countries_cluster_withPCA]
   :Scatter   [PCA1]   (PCA2,country-year)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [44]:
# Composite plot to contrast the Elbow curves

plots = (elbow_original_data + elbow_with_pca).cols(1)
plots

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [49]:
# Composite plot to contrast the clusters
# YOUR CODE HERE!


plots = (clusters_original_data + clusters_with_PCA).cols(1)
plots

:Layout
   .NdOverlay.I  :NdOverlay   [countries_cluster_original_data]
      :Scatter   [immigration_100k]   (Global_Peace_Index_2023,country-year)
   .NdOverlay.II :NdOverlay   [countries_cluster_withPCA]
      :Scatter   [PCA1]   (PCA2,country-year)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** 

  1)  regarding the number of clusters
  the fact that the optimal k was the same with and without use of PCA suggests the following : 
  - PCA did not significantly distort the structure of the data relevant to clustering..
  - The original dataset likely had a strong cluster structure and the clustering information was present in the original feature set.
  - the clusters in the original dataset are well separated, so the PCA did not change significantly the separability.

On other hand this consistancy is positive because 
  - it gives more confidence in the clustering results


  2) regarding the clustering assignements
  we observe that both methods (with and without PCA) give the same clusters (whatever is the # of the cluster)
  all the countries belong to the same cluster in both methods as shown in the dataframe below

  this reinforce the conclusions above

In [47]:
# to have the detailled content of the clusters, we will merge the 2 dataframes (with original data and with PCA)
# than we will compare the content of each cluster
# no need to make further queries, the data is limited and allows us to conclude

clusters = pd.merge(indicators_pca_new_df, indicators_prediction_df, on='country-year')
clusters = clusters[['countries_cluster_original_data', 'countries_cluster_withPCA']].sort_values(['countries_cluster_withPCA'])

# clusters.head(5)
clusters

,countries_cluster_original_data,countries_cluster_withPCA
country-year,,
Kuwait - 2018,0,0
"Hong Kong SAR, China - 2020",0,0
"Hong Kong SAR, China - 2021",0,0
"Hong Kong SAR, China - 2022",0,0
"Hong Kong SAR, China - 2023",0,0
...,...,...
Somalia - 2019,1,3
Somalia - 2018,1,3
Somalia - 2017,1,3


In [48]:
clusters.to_csv('Output_analysis/Clusters_analysis.csv')

### Mearge cluster information in full indicators dataframe

In [50]:
indicators_scaled_df.head(1)


,clean fuels for cooking (% of population),Access to electricity (% of population),Agricultural land (% ),Armed forces (% labor force),precipitation (mm per year),"Birth rate, crude (per 1,000 people)","Central government debt, total (% of GDP)",Consumer price index (2010 = 100),Control of Corruption: Estimate,Cost of business start-up (% of GNI per capita),...,"Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)",Total debt service (% of GNI),"Total debt service (% of exports of goods, services and primary income)",Unemployment advanced education,Unemployment basic education,Unemployment intermediate education,"Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,immigration_100k
country-year,,,,,,,,,,,,,,,,,,,,,
Afghanistan - 2015,-0.952155,-0.449773,0.88705,1.668181,-0.937583,1.911776,-0.599072,-0.09818,-1.247837,-0.071469,...,-1.263143,-0.538305,-0.477919,-0.952657,-0.898684,-1.019808,0.340125,-0.958168,1.626599,-0.099321


In [51]:
# Reset the index, moving 'country-year' back into a column
indicators_scaled_df.reset_index(inplace=True)
indicators_scaled_df.head(1)

,country-year,clean fuels for cooking (% of population),Access to electricity (% of population),Agricultural land (% ),Armed forces (% labor force),precipitation (mm per year),"Birth rate, crude (per 1,000 people)","Central government debt, total (% of GDP)",Consumer price index (2010 = 100),Control of Corruption: Estimate,...,"Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)",Total debt service (% of GNI),"Total debt service (% of exports of goods, services and primary income)",Unemployment advanced education,Unemployment basic education,Unemployment intermediate education,"Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,immigration_100k
0,Afghanistan - 2015,-0.952155,-0.449773,0.88705,1.668181,-0.937583,1.911776,-0.599072,-0.09818,-1.247837,...,-1.263143,-0.538305,-0.477919,-0.952657,-0.898684,-1.019808,0.340125,-0.958168,1.626599,-0.099321


In [52]:
new_df = pd.merge(indicators_scaled_df, clusters, on = 'country-year' )
new_df.columns

Index(['country-year', 'clean fuels for cooking (% of population)',
       'Access to electricity (% of population)', 'Agricultural land (% )',
       'Armed forces (% labor force)', 'precipitation (mm per year)',
       'Birth rate, crude (per 1,000 people)',
       'Central government debt, total (% of GDP)',
       'Consumer price index (2010 = 100)', 'Control of Corruption: Estimate',
       'Cost of business start-up (% of GNI per capita)',
       'Current account balance (% of GDP)',
       'Current health expenditure (% of GDP)',
       'Death rate, crude (per 1,000 people)',
       'Debt service (PPG and IMF only, % of exports of goods, services and primary income)',
       'doing business score', 'Fertility rate, total (births per woman)',
       '% Firms using banks', 'GDP growth (annual %)',
       'GDP per capita (constant 2015 US$)',
       'GDP per capita, PPP (constant 2021 international $)',
       'GNI per capita (2015 US$)', 'government  consumption exp (% of GDP)',
 

In [53]:
# to have the population of each cluster
x = new_df.groupby('countries_cluster_withPCA')['country-year'].count()
x

countries_cluster_withPCA
0    501
1    586
2    245
3    331
Name: country-year, dtype: int64

In [56]:
new_df.head(5)

,country-year,clean fuels for cooking (% of population),Access to electricity (% of population),Agricultural land (% ),Armed forces (% labor force),precipitation (mm per year),"Birth rate, crude (per 1,000 people)","Central government debt, total (% of GDP)",Consumer price index (2010 = 100),Control of Corruption: Estimate,...,"Total debt service (% of exports of goods, services and primary income)",Unemployment advanced education,Unemployment basic education,Unemployment intermediate education,"Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,immigration_100k,countries_cluster_original_data,countries_cluster_withPCA
0,Afghanistan - 2015,-0.952155,-0.449773,0.887050,1.668181,-0.937583,1.911776,-0.599072,-0.098180,-1.247837,...,-0.477919,-0.952657,-0.898684,-1.019808,0.340125,-0.958168,1.626599,-0.099321,1,3
1,Afghanistan - 2016,-0.921814,0.517416,0.887050,1.610880,-0.937583,1.826180,-0.599072,-0.095043,-1.430819,...,-0.404039,-0.952657,-0.898684,-1.019808,0.527347,-0.958168,1.626599,-0.102459,1,3
2,Afghanistan - 2017,-0.883888,0.517416,0.887050,1.524272,-0.937583,1.767536,-0.599072,-0.091327,-1.420804,...,-0.402597,1.688640,0.696312,1.421518,0.709374,-0.958168,1.626599,-0.042975,1,2
3,Afghanistan - 2018,-0.856076,0.358678,0.893859,1.175262,-0.937583,1.726564,-0.599072,-0.090836,-1.393977,...,-0.476190,1.688640,0.696312,1.421518,0.711452,-0.958168,1.626599,-0.043149,1,2
4,Afghanistan - 2019,-0.825735,0.517416,0.893859,1.215508,-0.937583,1.681050,-0.599072,-0.089020,-1.311552,...,-0.501171,1.688640,0.696312,1.421518,0.709547,-0.958168,1.626599,-0.026069,1,2


In [58]:
# Create a dictionary where each key is a cluster label and each value is the corresponding DataFrame
cluster_dfs = {cluster: group.copy() for cluster, group in new_df.groupby('countries_cluster_withPCA')}



In [61]:
df_cluster_0 = cluster_dfs[0]
df_cluster_0

,country-year,clean fuels for cooking (% of population),Access to electricity (% of population),Agricultural land (% ),Armed forces (% labor force),precipitation (mm per year),"Birth rate, crude (per 1,000 people)","Central government debt, total (% of GDP)",Consumer price index (2010 = 100),Control of Corruption: Estimate,...,"Total debt service (% of exports of goods, services and primary income)",Unemployment advanced education,Unemployment basic education,Unemployment intermediate education,"Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,immigration_100k,countries_cluster_original_data,countries_cluster_withPCA
75,Australia - 2015,0.87841,0.602321,0.318200,-0.439682,-0.67993,-0.645560,0.565421,-0.109399,1.904539,...,-0.713725,-0.346150,0.469977,-0.090426,-0.178937,1.655342,-0.357954,-0.145501,0,0
76,Australia - 2016,0.87841,0.602321,0.283806,-0.443782,-0.67993,-0.645560,0.770751,-0.108629,1.836426,...,-0.713725,-0.361526,0.410244,-0.126695,-0.238516,1.655342,-0.357954,-0.119409,0,0
77,Australia - 2017,0.87841,0.602321,0.452592,-0.449043,-0.67993,-0.675179,0.736640,-0.107438,1.817412,...,-0.713725,-0.387153,0.424528,-0.193187,-0.259126,1.655342,-0.357954,-0.103834,0,0
78,Australia - 2018,0.87841,0.602321,0.377872,-0.455116,-0.67993,-0.675179,0.766368,-0.106248,1.831695,...,-0.713725,-0.359134,0.377261,-0.175657,-0.309698,1.655342,-0.357954,-0.070976,0,0
79,Australia - 2019,0.87841,0.602321,0.398552,-0.456562,-0.67993,-0.724542,0.909745,-0.105225,1.852032,...,-0.713725,-0.394499,0.380637,-0.208752,-0.334119,1.655342,-0.357954,-0.079874,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,Uruguay - 2019,0.87841,0.598630,1.890458,0.085734,0.27351,-0.850716,0.656848,-0.060471,1.267448,...,-0.713725,-0.437724,0.469847,-0.071536,0.302715,1.560305,0.045052,-0.303730,0,0
1595,Uruguay - 2020,0.87841,0.602321,1.874244,0.114132,0.27351,-0.898401,0.911056,-0.049812,1.434407,...,-0.713725,-0.437724,0.469847,-0.071536,0.575842,1.560305,0.045052,-0.325149,0,0
1596,Uruguay - 2021,0.87841,0.602321,1.875918,0.114132,0.27351,-0.885172,0.911056,-0.040521,1.652683,...,-0.713725,-0.437724,0.469847,-0.071536,0.387926,1.560305,0.045052,-0.269452,0,0
1597,Uruguay - 2022,0.87841,0.602321,1.897355,0.114132,0.27351,-0.890503,0.911056,-0.028758,1.676885,...,-0.713725,-0.677935,0.357263,0.113433,0.136622,1.560305,0.045052,-0.282232,0,0


In [62]:
df_cluster_1 = cluster_dfs[1]
df_cluster_1

,country-year,clean fuels for cooking (% of population),Access to electricity (% of population),Agricultural land (% ),Armed forces (% labor force),precipitation (mm per year),"Birth rate, crude (per 1,000 people)","Central government debt, total (% of GDP)",Consumer price index (2010 = 100),Control of Corruption: Estimate,...,"Total debt service (% of exports of goods, services and primary income)",Unemployment advanced education,Unemployment basic education,Unemployment intermediate education,"Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,immigration_100k,countries_cluster_original_data,countries_cluster_withPCA
9,Albania - 2015,0.286763,0.602321,0.209097,-0.312831,0.503780,-0.762256,1.401735,-0.109748,-0.449321,...,1.682417,2.340593,0.854344,2.078432,1.750095,-0.150356,-0.049773,0.245703,2,1
10,Albania - 2016,0.329746,0.598630,0.221091,-0.327565,0.503780,-0.804610,1.423560,-0.108983,-0.377547,...,0.375061,1.960971,0.778510,1.647743,1.442676,-0.150356,-0.049773,0.190958,2,1
11,Albania - 2017,0.362616,0.598630,0.209067,-0.310344,0.503780,-0.845977,1.302134,-0.107776,-0.386437,...,0.082600,1.386924,0.690728,1.291405,1.130581,-0.150356,-0.049773,0.333240,2,1
12,Albania - 2018,0.390428,0.602321,0.208743,-0.319639,0.503780,-0.880828,0.995116,-0.106520,-0.450187,...,0.763892,1.441595,0.387779,1.130917,0.903351,-0.150356,-0.049773,0.329789,2,1
13,Albania - 2019,0.423297,0.602321,0.208611,-0.371779,0.503780,-0.898006,1.275066,-0.105628,-0.468193,...,0.115270,1.148080,0.229357,1.134997,0.758215,-0.150356,-0.049773,0.307092,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631,West Bank and Gaza - 2019,-1.649995,0.602321,1.443441,-0.736589,-0.844231,0.989958,-0.599072,-0.109145,-0.260550,...,-0.713725,4.250664,2.023809,2.820728,3.161105,-1.908535,1.150216,0.038257,2,1
1632,West Bank and Gaza - 2020,-1.649995,0.602321,1.314612,-0.736589,-0.844231,0.924206,-0.599072,-0.109591,-0.480025,...,-0.713725,4.067516,2.258974,2.771312,3.257228,-1.908535,1.150216,-0.037323,2,1
1633,West Bank and Gaza - 2021,-1.649995,0.602321,1.189291,-0.736589,-0.844231,0.868524,-0.599072,-0.108846,-0.662694,...,-0.713725,4.493780,2.101461,2.993911,3.342959,-1.908535,1.150216,0.254269,2,1
1634,West Bank and Gaza - 2022,-1.649995,0.602321,1.189291,-0.736589,-0.844231,0.814618,-0.599072,-0.106569,-0.604736,...,-0.713725,4.097415,1.906550,2.608406,3.001767,-1.908535,1.150216,0.176906,2,1


In [64]:
df_cluster_2 = cluster_dfs[2]
df_cluster_2

,country-year,clean fuels for cooking (% of population),Access to electricity (% of population),Agricultural land (% ),Armed forces (% labor force),precipitation (mm per year),"Birth rate, crude (per 1,000 people)","Central government debt, total (% of GDP)",Consumer price index (2010 = 100),Control of Corruption: Estimate,...,"Total debt service (% of exports of goods, services and primary income)",Unemployment advanced education,Unemployment basic education,Unemployment intermediate education,"Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,immigration_100k,countries_cluster_original_data,countries_cluster_withPCA
2,Afghanistan - 2017,-0.883888,0.517416,0.887050,1.524272,-0.937583,1.767536,-0.599072,-0.091327,-1.420804,...,-0.402597,1.688640,0.696312,1.421518,0.709374,-0.958168,1.626599,-0.042975,1,2
3,Afghanistan - 2018,-0.856076,0.358678,0.893859,1.175262,-0.937583,1.726564,-0.599072,-0.090836,-1.393977,...,-0.476190,1.688640,0.696312,1.421518,0.711452,-0.958168,1.626599,-0.043149,1,2
4,Afghanistan - 2019,-0.825735,0.517416,0.893859,1.215508,-0.937583,1.681050,-0.599072,-0.089020,-1.311552,...,-0.501171,1.688640,0.696312,1.421518,0.709547,-0.958168,1.626599,-0.026069,1,2
5,Afghanistan - 2020,-0.795394,0.517416,0.914490,0.439168,-0.937583,1.640079,-0.599072,-0.089020,-1.384592,...,-0.522630,1.504809,0.543084,0.879606,0.800474,-0.958168,1.626599,-0.201856,1,2
6,Afghanistan - 2021,-0.767581,0.517416,0.914490,0.439168,-0.937583,1.619444,-0.599072,-0.089020,-1.048155,...,-0.522630,0.718399,-0.021650,0.321977,0.849661,-0.958168,1.626599,0.298064,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1658,Zimbabwe - 2019,-0.883888,-1.365280,0.060801,-0.166245,-0.526832,1.192547,-0.599072,0.053540,-1.184337,...,1.447178,0.006650,-0.053854,0.359152,0.049332,-0.768094,0.612874,-0.218420,3,2
1659,Zimbabwe - 2020,-0.878831,-1.143787,0.071269,-0.170225,-0.526832,1.142295,-0.599072,1.297569,-1.202539,...,0.621360,0.006650,-0.053854,0.359152,0.265478,-0.768094,0.612874,-0.262353,3,2
1660,Zimbabwe - 2021,-0.878831,-1.280374,0.054919,-0.170225,-0.526832,1.095695,-0.599072,2.743528,-1.171331,...,-0.054160,0.263091,0.151705,0.731509,0.424643,-0.768094,0.612874,-0.232106,1,2
1661,Zimbabwe - 2022,-0.871246,-1.239767,0.059510,-0.170225,-0.526832,1.051169,-0.599072,5.793854,-1.154388,...,-0.278601,0.063029,0.263769,0.858147,0.519380,-0.768094,0.612874,-0.213756,3,2


In [65]:
df_cluster_3 = cluster_dfs[3]
df_cluster_3

,country-year,clean fuels for cooking (% of population),Access to electricity (% of population),Agricultural land (% ),Armed forces (% labor force),precipitation (mm per year),"Birth rate, crude (per 1,000 people)","Central government debt, total (% of GDP)",Consumer price index (2010 = 100),Control of Corruption: Estimate,...,"Total debt service (% of exports of goods, services and primary income)",Unemployment advanced education,Unemployment basic education,Unemployment intermediate education,"Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,immigration_100k,countries_cluster_original_data,countries_cluster_withPCA
0,Afghanistan - 2015,-0.952155,-0.449773,0.887050,1.668181,-0.937583,1.911776,-0.599072,-0.098180,-1.247837,...,-0.477919,-0.952657,-0.898684,-1.019808,0.340125,-0.958168,1.626599,-0.099321,1,3
1,Afghanistan - 2016,-0.921814,0.517416,0.887050,1.610880,-0.937583,1.826180,-0.599072,-0.095043,-1.430819,...,-0.404039,-0.952657,-0.898684,-1.019808,0.527347,-0.958168,1.626599,-0.102459,1,3
18,Algeria - 2015,0.868297,0.580172,-0.921197,0.962167,-1.233822,0.589026,-0.599072,-0.101489,-0.542354,...,-0.590560,-0.952657,-0.898684,-1.019808,0.713184,-0.197874,0.290017,-0.119329,2,3
19,Algeria - 2016,0.868297,0.580172,-0.922990,0.943468,-1.233822,0.565430,-0.599072,-0.097123,-0.576125,...,-0.641257,-0.952657,-0.898684,-1.019808,0.539298,-0.197874,0.290017,-0.124585,2,3
27,American Samoa - 2015,-1.649995,-3.089239,-1.090215,-0.736589,-1.344600,-1.919144,-0.599072,-0.169723,1.224377,...,-0.713725,-0.952657,-0.898684,-1.019808,-1.227625,-1.908535,-2.091898,-0.325149,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1642,"Yemen, Rep. - 2021",-0.395906,-0.324260,0.278444,-0.361699,-1.136735,1.096387,-0.599072,-0.169723,-1.557514,...,0.327016,-0.952657,-0.898684,-1.019808,1.939569,-1.148241,1.742872,-0.216612,1,3
1643,"Yemen, Rep. - 2022",-0.428775,-0.283653,0.278444,-0.361699,-1.136735,1.034682,-0.599072,-0.169723,-1.568498,...,0.327016,-0.952657,-0.898684,-1.019808,1.779538,-1.148241,1.742872,-0.262709,1,3
1644,"Yemen, Rep. - 2023",-0.428775,-0.283653,0.278444,-0.361699,-1.136735,1.034682,-0.599072,-0.169723,-1.538040,...,0.327016,-0.952657,-0.898684,-1.019808,1.732430,-1.148241,1.742872,-0.276752,1,3
1654,Zimbabwe - 2015,-0.891473,-1.845183,0.078548,-0.124277,-0.526832,1.433639,-0.599072,-0.112539,-1.231131,...,0.486887,-0.952657,-0.898684,-1.019808,-0.296362,-0.768094,0.612874,-0.278658,1,3


### Define the target and predictors.

In [72]:
# Select features and target
target = 'immigration_100k'
X = df_cluster_0.drop(columns= [target, 'country-year','countries_cluster_original_data','countries_cluster_withPCA'])
y = df_cluster_0[target]

In [73]:
X.shape

(501, 48)

In [74]:
y.shape

(501,)

## Splitting the data

In [75]:
## Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [76]:
X_train.shape

(400, 48)

In [77]:
X_test.shape

(101, 48)

## Regression Modelling

### --- Linear Regression ---

In [80]:
# --- Linear Regression ---
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
mse_lr = mean_squared_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)
print("Linear Regression MSE:", mse_lr)
print("Linear Regression R2:", r2_lr)

Linear Regression MSE: 0.054266712759710935
Linear Regression R2: 0.3125801486108165


### --- Random Forest Regression ---

In [81]:
### --- Random Forest Regression ---
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
print("Random Forest Regression MSE:", mse_rf)
print("Random Forest Regression R2:", r2_rf)

Random Forest Regression MSE: 0.00906833260423191
Random Forest Regression R2: 0.8851275204608139


The results show that the Random Forest model significantly outperforms the Linear Regression model:

- Mean Squared Error (MSE):

  - Linear Regression: 0.054
  - Random Forest: 0.009
   A lower MSE indicates that the Random Forest model's predictions are, on average, much closer to the actual values.

- R² Score:

  - Linear Regression: 0.31 (approximately 31% of the variance explained)
  - Random Forest: 0.885 (approximately 88.5% of the variance explained)
   The higher R² score of the Random Forest model means it captures a significantly larger proportion of the variability in immigration_100k.
   
In summary, the Random Forest model is more accurate and reliable for predicting immigration per 100k, likely due to its ability to capture non-linear relationships and interactions among the predictors that a simple linear model might miss.